In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

raw_csv=np.loadtxt('Audiobooks-data.csv',delimiter=',')

In [5]:
unscaled_input=raw_csv[:,1:-1]

target_all=raw_csv[:,-1]


BALANCING THE DATASET

In [6]:
num_one_target=int(np.sum(target_all))
zero_target=0
indices_to_remove=[]
for i in range(target_all.shape[0]):#shape0 gives shape of the vector
    if target_all[i]==0:
        zero_target+=1
        if zero_target>num_one_target:
            indices_to_remove.append(i)
            

In [7]:
unscaled_input_equal=np.delete(unscaled_input,indices_to_remove,axis=0)
target_equal=np.delete(target_all,indices_to_remove,axis=0)

STANDARDISE THE inputs 

In [8]:
scalar_deep_learning=StandardScaler()#
scaled_inputs=scalar_deep_learning.fit_transform(unscaled_input_equal)#to tranform in one line

SHUFFLE DATA aince we are batching because maybe data is 
a little heterogenous if it is placed according to dates

In [9]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=target_equal[shuffled_indices]#so that input still aligned by target

SPLIT DATASET INTO TRAIN VALIDATION TEST

In [10]:
samples_count=shuffled_inputs.shape[0]
train_sample=int(0.8*samples_count)
validation_sample=int(0.1*samples_count)
test_sample=samples_count-train_sample-validation_sample

EXTRACT THE TEST VAL TEST from big dataset

In [11]:
train_input=shuffled_inputs[:train_sample]
train_target=shuffled_targets[:train_sample]

validation_input=shuffled_inputs[train_sample:train_sample+validation_sample]
validation_target=shuffled_targets[train_sample:train_sample+validation_sample]

test_input=shuffled_inputs[train_sample+validation_sample:]
test_target=shuffled_targets[train_sample+validation_sample:]

IN NPZ

In [12]:
np.savez('Audiobooks_data_train',inputs=train_input ,targets=train_target)
np.savez('Audiobooks_data_validation',inputs=validation_input ,targets=validation_target)
np.savez('Audiobooks_data_test',inputs=test_input ,targets=test_target)

In [13]:
#SAVe THE SCALAR
pickle.dump(scalar_deep_learning,open('scalar_deep_learning.pickle','wb'))

In [15]:
import tensorflow as tf

In [16]:
npz=np.load('Audiobooks_data_train.npz')
train_inputs=npz['inputs'].astype(np.float)
train_targets=npz['targets'].astype(np.int)

npz=np.load('Audiobooks_data_validation.npz')
validation_inputs=npz['inputs'].astype(np.float)
validation_targets=npz['targets'].astype(np.int)

npz=np.load('Audiobooks_data_test.npz')
test_inputs=npz['inputs'].astype(np.float)
test_targets=npz['targets'].astype(np.int)

D:\anacondaa3\envs\py3-TF2.0\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
D:\anacondaa3\envs\py3-TF2.0\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#

In [20]:
# automatically detect input nodesinput_size=10
output_size=2#0 or 1
hidden_layer=50
model=tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer,activation='relu'),
    tf.keras.layers.Dense(hidden_layer,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
    
    
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
batch_size=100
max_epochs=100
early=tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(
    train_inputs,
    train_targets,
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early],
    validation_data=(validation_input,validation_target),
    verbose=2
)

Epoch 1/100
36/36 - 3s - loss: 0.5702 - accuracy: 0.6946 - val_loss: 0.4706 - val_accuracy: 0.8233 - 3s/epoch - 80ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4515 - accuracy: 0.7709 - val_loss: 0.4089 - val_accuracy: 0.7651 - 93ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4049 - accuracy: 0.7843 - val_loss: 0.3760 - val_accuracy: 0.7763 - 126ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3850 - accuracy: 0.7930 - val_loss: 0.3554 - val_accuracy: 0.8210 - 117ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3708 - accuracy: 0.8025 - val_loss: 0.3449 - val_accuracy: 0.8233 - 109ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3627 - accuracy: 0.8106 - val_loss: 0.3427 - val_accuracy: 0.8188 - 107ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3573 - accuracy: 0.8092 - val_loss: 0.3367 - val_accuracy: 0.8210 - 114ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3525 - accuracy: 0.8075 - val_loss: 0.3403 - val_accuracy: 0.8166 - 149ms/epoch - 4ms/step
Epoch 9/100

In [22]:
TEST THE MODEL

SyntaxError: invalid syntax (1054046285.py, line 1)

In [21]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)


14/14 [==============================] - 0s 2ms/step - loss: 0.3412 - accuracy: 0.8259


OBTAIN THE PROBABILTY FOR CUSTOMER TO CONVERT

In [24]:
model.predict(test_inputs)

14/14 [==============================] - 0s 2ms/step


array([[9.99877214e-01, 1.22761645e-04],
       [8.88694369e-04, 9.99111176e-01],
       [1.14180043e-03, 9.98858094e-01],
       [2.12305397e-01, 7.87694514e-01],
       [4.06806290e-01, 5.93193769e-01],
       [1.06782429e-01, 8.93217564e-01],
       [1.72886681e-02, 9.82711315e-01],
       [1.14180043e-03, 9.98858094e-01],
       [3.51018488e-01, 6.48981571e-01],
       [4.11273181e-01, 5.88726819e-01],
       [1.81358645e-03, 9.98186409e-01],
       [9.99988377e-01, 1.15164867e-05],
       [9.85493720e-01, 1.45063270e-02],
       [5.22648871e-01, 4.77351129e-01],
       [9.99873400e-01, 1.26732222e-04],
       [7.85992563e-01, 2.14007393e-01],
       [4.22834694e-01, 5.77165306e-01],
       [2.43439049e-01, 7.56560981e-01],
       [4.01547432e-01, 5.98452568e-01],
       [5.90669513e-01, 4.09330487e-01],
       [1.11104943e-01, 8.88895094e-01],
       [3.60676706e-01, 6.39323235e-01],
       [2.94363737e-01, 7.05636203e-01],
       [9.50167835e-01, 4.98322211e-02],
       [1.826057

In [29]:
model.predict(test_inputs)[:,1].round(0)#only works for 2 classes 

14/14 [==============================] - 0s 2ms/step


array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1.

In [30]:
np.argmax(model.predict(test_inputs),axis=1)

14/14 [==============================] - 0s 1ms/step


array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,

In [ ]:
SAVE

In [31]:
model.save('audiobooks_model.h5')

PREDICTING ON NEW DATA

import relevant libraries